<a href="https://colab.research.google.com/github/KosarDst/Anngroup-Ui/blob/main/point_center_algorithm_for_k_means_clustering_to_increase_software_defect_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 2: Calculate the point center algorithm to get 𝑘 value and point center


In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/CM1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/KC1.csv')
df3 = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/KC3.csv')
df4 = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/MC2.csv')
df5 = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/MW1.csv')
df6 = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/PC1.csv')
df7 = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/PC2.csv')
df8 = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/PC3.csv')
df9 = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/PC4.csv')

# combine dataframes into one
# df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], ignore_index=True)
# df = df.drop(df.index[0])
# df = df.replace(to_replace=r'^\D*$', value=0, regex=True)
# df = df.apply(pd.to_numeric)

# Simple K-means

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

# Load the CSV file into a Pandas DataFrame
df =pd.read_csv('/content/drive/MyDrive/Dataset_SDP/PC2.csv')

# Remove the 'id' and 'Defective' columns from the DataFrame
X = df.drop(['Defective'], axis=1)

# Fit the K-Means model with k=2
kmeans = KMeans(n_clusters=2).fit(X)

# Predict the cluster labels
labels = kmeans.predict(X)

# Convert 'N' and 'Y' labels to integers
label_map = {'N': 0, 'Y': 1}
y_true = df['Defective'].map(label_map)

# Create a confusion matrix
cm = confusion_matrix(y_true, labels)

# # Extract the True Negative, False Positive, False Negative, and True Positive values
tn, fn, fp, tp = cm.ravel()

# Print the results
print('True Negative:', tn)
print('False Positive:', fp)
print('False Negative:', fn)
print('True Positive:', tp)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


True Negative: 1465
False Positive: 15
False Negative: 12
True Positive: 1


# Proposed K-means

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/PC2.csv')

# Identify string columns
string_cols = df.select_dtypes(include='object').columns

# Replace string values with numeric
df[string_cols] = df[string_cols].replace(['N', 'Y'], [0,1])

# Replace missing values with mean
df = df.fillna(df.mean())

# Calculate point center
means = df.mean()
k = len(df['Defective'].unique())
centroids = [means] * k

# Perform k-means clustering
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=k, init=centroids, n_init=1)
clusters = kmeans.fit_predict(df)

# Calculate error and Rand index
confusion_matrix = pd.crosstab(df['Defective'], clusters)
print(confusion_matrix)
error = (confusion_matrix.iloc[0,1] + confusion_matrix.iloc[1,0]) / confusion_matrix.sum().sum()
rand_index = (confusion_matrix.iloc[0,0] + confusion_matrix.iloc[1,1]) / confusion_matrix.sum().sum()

print(f"Error: {error}")
print(f"Rand Index: {rand_index}")

col_0         0  1
Defective         
0          1470  7
1            15  1
Error: 0.014735432016075016
Rand Index: 0.985264567983925


# point center algorithm

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Dataset_SDP/PC3.csv')
data = data.replace(to_replace=r'^\D*$', value=0, regex=True)
# Calculate the mean and standard deviation for each attribute
means = data.mean()
stds = data.std()

# Determine the center data point based on the maximum average of standard deviation
center = means + stds
center_point = center.idxmax()

# Calculate the Euclidean distance of each data point from the center point
distances = np.sqrt(((data - center) ** 2).sum(axis=1))

# Select the data point with the maximum distance as the first candidate center point (c1)
c1 = data.iloc[distances.idxmax()]

# Select variables X and Y based on minimum average and maximum standard deviation
min_mean = means.min()
max_std = stds.max()
X = means[means == min_mean].index[0]
Y = stds[stds == max_std].index[0]

# Calculate the Euclidean distance from c1 and select the point with maximum distance as the second candidate center point (c2)
distances_X = np.sqrt(((data[X] - c1[X]) ** 2))
distances_Y = np.sqrt(((data[Y] - c1[Y]) ** 2))
distances = np.sqrt(distances_X ** 2 + distances_Y ** 2)
c2 = data.iloc[distances.idxmax()]

# Continue this process until the distance equals a previous distance, giving you the number of clusters k
prev_distance = 0
k = 2
while True:
    distances_X = np.sqrt(((data[X] - c1[X]) ** 2))
    distances_Y = np.sqrt(((data[Y] - c1[Y]) ** 2))
    distances_c1 = np.sqrt(distances_X ** 2 + distances_Y ** 2)
    distances_X = np.sqrt(((data[X] - c2[X]) ** 2))
    distances_Y = np.sqrt(((data[Y] - c2[Y]) ** 2))
    distances_c2 = np.sqrt(distances_X ** 2 + distances_Y ** 2)
    distances = pd.concat([distances_c1, distances_c2], axis=1).min(axis=1)
    distance = distances.sum()
    if distance == prev_distance:
        break
    prev_distance = distance
    k += 1
    new_c = data.iloc[distances.idxmax()]
    c1 = (c1 * (k - 1) + new_c) / k

# The candidate center points c1, c2, ... , ck are the initial centroids for K-Means clustering
initial_centroids = pd.concat([c1, c2], axis=1).T.to_numpy()

print("Center point:", center_point)
print("Number of clusters (k):", k)
print("Initial centroids:")
print(initial_centroids)

**proposed method / not completed**

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, adjusted_rand_score

# Step 1: Initialize the K value and point center value
k = 2
point_center = 3

# Step 2: Load the dataset and perform KMeans clustering
data = pd.read_csv('CM1.csv')
X = data.drop('Defective', axis=1)
kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
kmeans.fit(X)

# Step 3: Evaluate the KMeans clustering using the proposed algorithm
y_kmeans = kmeans.labels_
y_proposed = [1 if x >= point_center else 0 for x in kmeans.predict(X)]
cm = confusion_matrix(data['Defective'], y_proposed)
error_rate = 1 - accuracy_score(data['Defective'], y_proposed)
rand_index = adjusted_rand_score(data['Defective'], y_proposed)

# Print the results
print("Confusion Matrix:\n", cm)
print("Error Rate:", error_rate)
print("Rand Index:", rand_index)